In [114]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [115]:
url = 'https://www.billboard.com/charts/hot-100/'

response = requests.get(url)
soup = BS(response.content, 'html.parser')

print(type(response))
print(response.status_code)
print(requests.get(url))

<class 'requests.models.Response'>
200
<Response [200]>


In [46]:
first = soup.find('li', class_ = 'o-chart-results-list__item').text.strip()
first

'1'

In [37]:
song = soup.findAll('h3', id = 'title-of-a-story')[2].text.strip()
song

'Golden'

In [42]:
artist = soup.findAll('span', class_ = 'c-label')[1].text.strip()
artist

'HUNTR/X: EJAE, Audrey Nuna & REI AMI'

In [57]:
stats = soup.find_all('span', class_ = 'c-label')
lastweek = stats[2].text.strip()
peak = stats[3].text.strip()
weeks_chart = stats[4].text.strip()

print(lastweek)
print(peak)
print(weeks_chart)

1
1
14


#### Start with isolating #1 on this weeks chart

In [116]:
data=[]

# First song (rank 1) has a unique structure compared to 2-100
first = soup.find('li', class_ = 'o-chart-results-list__item').text.strip()
song = soup.findAll('h3', id = 'title-of-a-story')[2].text.strip()
artist = soup.findAll('span', class_ = 'c-label')[1].text.strip()
stats = soup.findAll('span', class_ = 'c-label')
lastweek = stats[2].text.strip()
peak = stats[3].text.strip()
weeks_chart = stats[4].text.strip()


data.append({
    "This Week": first,
    "Song": song,
    "Artist": artist,
    "Last Week": lastweek,
    "Peak Position": peak,
    "Weeks on Chart": weeks_chart
})

data = pd.DataFrame(data)
data

,This Week,Song,Artist,Last Week,Peak Position,Weeks on Chart
0,1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",1,1,14


#### Now create a loop for extracting 2-100

In [182]:
forloop = soup.find_all('ul', class_='o-chart-results-list-row')
forloop

# Rank = ul , class_ = o-chart-results-list-row
# Title = h3 , class_ = c-title
# Artist = span , class_ = c-label a-no-tucate
# Stats = span , class_ = c-label
# Last Week = [-3]
# Peak = [-2]
# Weeks on Chart = [-1]

[<ul class="o-chart-results-list-row // lrv-a-unstyle-list lrv-u-flex lrv-u-background-color-white a-chart-has-chart-detail u-border-radius-a-6 u-border-radius-a-0@mobile-max lrv-u-margin-b-1 u-margin-b-0.625@mobile-max lrv-u-padding-tb-050 lrv-u-align-items-center lrv-u-padding-r-050@mobile-max lrv" data-ajax="" data-detail-target="1">
 <li class="o-chart-results-list__item // lrv-u-color-black u-width-60 u-width-45px@mobile-max lrv-u-flex lrv-u-flex-direction-column lrv-u-flex-shrink-0 lrv-u-align-items-center lrv-u-justify-content-center u-height-60 lrv-u-background-color-brand-primary@desktop">
 <span class="c-label a-font-basic u-font-size-33@desktop u-font-size-19@mobile-max u-line-height-33px">
 	
 	1
 </span>
 <div class="c-svg u-height-10@mobile-max u-width-10@mobile-max u-hidden@tablet">
 <svg viewbox="0 0 504.22 417.06" xmlns="http://www.w3.org/2000/svg"><path d="m490.74 178.78.04-.04L321.05 9l-.09.09C314.47 3.45 306 0 296.72 0c-20.43 0-37 16.57-37 37 0 9.28 3.44 17.74 9.09 

In [194]:
data=[]

# # First song (rank 1) has a unique structure compared to 2-100
# first = soup.find('li', class_ = 'o-chart-results-list__item').text.strip()
# song = soup.findAll('h3', id = 'title-of-a-story')[2].text.strip()
# artist = soup.findAll('span', class_ = 'c-label')[1].text.strip()
# stats = soup.findAll('span', class_ = 'c-label')
# lastweek = stats[2].text.strip()
# peak = stats[3].text.strip()
# weeks_chart = stats[4].text.strip()


# data.append({
#     "This Week": first,
#     "Song": song,
#     "Artist": artist,
#     "Last Week": lastweek,
#     "Peak Position": peak,
#     "Weeks on Chart": weeks_chart
# })

forloop = soup.find_all('ul', class_='o-chart-results-list-row') # [1:] skips the first song if the code above is used. The loop works for 1-100 without using the above code. 

for row in forloop:
    try:
        # Get all li items in this row
        items = row.find_all('li', class_='o-chart-results-list__item')
        
        # Rank is in the first li
        rank = items[0].find('span', class_='c-label').text.strip()
        
        # Song title 
        songtitle = row.find('h3', class_='c-title')
        song = songtitle.text.strip()
        
        # I have not found a way to bypass the fallback. It appears to be absolutely necessary to correctly extract the artist/s name.
        artist_elem = row.find('span', class_='c-label a-no-trucate')
        if not artist_elem:
            artist_list = row.find('ul', class_='lrv-a-unstyle-list')
            if artist_list:
                artist_elem = artist_list.find('span')
        artist = artist_elem.text.strip() if artist_elem else 'N/A'
        
        stats = row.find_all('span', class_ ='c-label')
        
        # Last 3 stats are typically: last week, peak, weeks on chart
        # if len(stats) >= x else 'N/A' can be added as a safety check
        lastweek = stats[-3].text.strip()
        peak = stats[-2].text.strip()
        weeks_chart = stats[-1].text.strip()
        
        data.append({
            "This Week": rank,
            "Song": song,
            "Artist": artist,
            "Last Week": lastweek,
            "Peak Position": peak,
            "Weeks on Chart": weeks_chart
        })
    except Exception as e:
        print(f"Error processing row: {e}") # Handles errors, if error will continue to next song
        continue

print(f"Scraped {len(data)} songs") # Returns how many songs were scraped

df = pd.DataFrame(data)
df.head(20)

Scraped 100 songs


,This Week,Song,Artist,Last Week,Peak Position,Weeks on Chart
0,1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",1,1,14
1,2,Ordinary,Alex Warren,2,1,33
2,3,Soda Pop,"Saja Boys: Andrew Choi, Neckwav, Danny Chung, ...",5,3,13
3,4,What I Want,Morgan Wallen Featuring Tate McRae,4,1,19
4,5,Your Idol,"Saja Boys: Andrew Choi, Neckwav, Danny Chung, ...",6,4,14
5,6,Manchild,Sabrina Carpenter,3,1,16
6,7,Lose Control,Teddy Swims,9,1,110
7,8,Daisies,Justin Bieber,8,2,11
8,9,Love Me Not,Ravyn Lenae,7,5,26
9,10,Die With A Smile,Lady Gaga & Bruno Mars,11,1,58


#### Found a pandas library for billboard

In [135]:
!pip install billboard.py pandas
import billboard, pandas as pd

chart = billboard.ChartData('hot-100')   # or 'billboard-200', etc.

data = [
    {
        "This Week Pos": entry.rank,
        "Song": entry.title,
        "Artist": entry.artist,
        "Last Week": entry.lastPos or None,
        "Peak Position": entry.peakPos,
        "Weeks on Chart": entry.weeks
    }
    for entry in chart
]

df = pd.DataFrame(data)

In [134]:
df.head()

,This Week Pos,Song,Artist,Last Week,Peak Position,Weeks on Chart
0,1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",1.0,1,14
1,2,Ordinary,Alex Warren,2.0,1,33
2,3,Soda Pop,"Saja Boys: Andrew Choi, Neckwav, Danny Chung, ...",5.0,3,13
3,4,What I Want,Morgan Wallen Featuring Tate McRae,4.0,1,19
4,5,Your Idol,"Saja Boys: Andrew Choi, Neckwav, Danny Chung, ...",6.0,4,14


In [139]:
df['Last Week'] = df['Last Week'].astype('Int64')
df.head()

,This Week Pos,Song,Artist,Last Week,Peak Position,Weeks on Chart
0,1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",1,1,14
1,2,Ordinary,Alex Warren,2,1,33
2,3,Soda Pop,"Saja Boys: Andrew Choi, Neckwav, Danny Chung, ...",5,3,13
3,4,What I Want,Morgan Wallen Featuring Tate McRae,4,1,19
4,5,Your Idol,"Saja Boys: Andrew Choi, Neckwav, Danny Chung, ...",6,4,14
